In [2]:
import matplotlib.pyplot as plt

In [3]:
import sqlalchemy

In [4]:
sqlalchemy.create_engine('sqlite:///vivino_v2.db')

Engine(sqlite:///vivino_v2.db)

In [5]:
%load_ext sql

In [6]:
%sql sqlite:///vivino_v2.db

We want to highlight 10 wines to increase our sales, which ones should we choose and why?

In [7]:
%%sql -- DONE
SELECT
    name,
    ratings_average AS rating, ratings_count
FROM
    wines
ORDER BY
    ratings_count DESC,rating DESC
LIMIT
    10

 * sqlite:///vivino_v2.db
Done.


name,rating,ratings_count
Cabernet Sauvignon,4.6,157944
Brut Champagne,4.6,146377
Tignanello,4.5,142513
Sassicaia,4.6,107646
60 Sessantanni Old Vines Primitivo di Manduria,4.5,94289
Costasera Amarone della Valpolicella Classico,4.3,93397
ARTEMIS Cabernet Sauvignon,4.4,81351
Amarone della Valpolicella Classico,4.3,77515
Opus One,4.6,77053
Pauillac (Premier Grand Cru Classé),4.6,73615


We have a marketing budget for this year, which country should we prioritise and why?

In [8]:
%%sql --DONE
SELECT
    *
FROM
    countries
ORDER BY
    users_count DESC
    
LIMIT
    3

 * sqlite:///vivino_v2.db
Done.


code,name,regions_count,users_count,wines_count,wineries_count
us,États-Unis,362,12273684,204060,28145
fr,France,1306,5973301,422503,67553
it,Italie,563,4270717,274658,42399


We would like to give a price to the best winery, which one should we choose and why?

In [9]:
%%sql --Complete
SELECT wineries.id, wineries.name, wines.ratings_average, SUM(wines.ratings_count)
FROM wineries
JOIN wines
ON wines.winery_id = wineries.id
GROUP BY wineries.name
ORDER BY wines.ratings_count DESC
LIMIT 3

 * sqlite:///vivino_v2.db
Done.


id,name,ratings_average,SUM(wines.ratings_count)
1301,Caymus,4.6,199180
212823,Dom Pérignon,4.6,167667
1413,Tenuta San Guido,4.6,107646


We has detected that a big cluster of customers like a specific combination of tastes. We have identified few primary keywords that matches this and we would like you to find all the wines that have those keywords. To ensure accuracy of our selection, ensure that more than 10 users confirmed those keywords. Also, identify the flavor_groups related to those keywords.

In [10]:
%%sql--TEST
SELECT wines.id, keywords.name as keyword, wines.name as wine, keywords_wine.keyword_type
FROM wines
JOIN keywords_wine
ON wines.id = keywords_wine.wine_id
JOIN keywords
ON keywords.id = keywords_wine.keyword_id
WHERE keywords.name IN ('coffee','toast','green apple', 'cream', 'citrus') AND (keywords_wine.count > 10) AND keywords_wine.keyword_type = 'primary'
LIMIT 5
--117,417,195,123,106

UsageError: Cell magic `%%sql--TEST` not found.


In [11]:
%%sql
SELECT wines.name AS wine,wines.id as wine_id, keywords_wine.group_name
FROM wines
JOIN keywords_wine ON wines.id = keywords_wine.wine_id
JOIN keywords ON keywords.id = keywords_wine.keyword_id
WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
AND keywords_wine.keyword_type = 'primary'
GROUP BY wines.id, wines.name
HAVING COUNT(keywords.id) = 5
AND keywords_wine.count > 10

 * sqlite:///vivino_v2.db
Done.


wine,wine_id,group_name
Tignanello,1652,oak
Tenuta Guado al Tasso Bolgheri Superiore,1658,oak
Monte Bello,3715,oak
Pauillac (Premier Grand Cru Classé),3908,oak
Sassicaia,5078,oak
Chateau Buena Vista Cabernet Sauvignon Napa Valley,5261,oak
Pavillon Rouge du Château Margaux,6020,oak
Amarone della Valpolicella Classico,7103,oak
Brut Cuvée Champagne Rosé,8305,microbio
Grand Vin de Château Palmer (Grand Cru Classé),9220,oak


In [12]:
%%sql--TESTS
SELECT wines.name AS wine,wines.id as wine_id, COUNT(keywords.id) AS keyword_count, keywords.name
FROM wines
JOIN keywords_wine ON wines.id = keywords_wine.wine_id
JOIN keywords ON keywords.id = keywords_wine.keyword_id
WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
  AND keywords_wine.keyword_type = 'primary'
GROUP BY wines.id, wines.name
HAVING COUNT(keywords.id) = 5
AND keywords_wine.count > 10
   

UsageError: Cell magic `%%sql--TESTS` not found.


In [13]:
%%sql -- TESTS
SELECT  keywords.name as keyword, wines.name as wine
FROM keywords
JOIN wines
ON wines.id = keywords_wine.wine_id
JOIN keywords
ON keywords.id = keywords_wine.keyword_id
--WHERE keywords.name IN ('coffee','toast','green apple', 'cream', 'citrus') AND (keywords_wine.count > 10)
--Adding a lot of ands, so wines have ALL the keywords
--WHERE (keywords.name = 'coffee') AND (keywords.name = 'toast') AND (keywords.name = 'green apple') AND (keywords.name = 'cream') AND (keywords.name = 'citrus')

 * sqlite:///vivino_v2.db
(sqlite3.OperationalError) ambiguous column name: keywords.name
[SQL: SELECT  keywords.name as keyword, wines.name as wine
FROM keywords
JOIN wines
ON wines.id = keywords_wine.wine_id
JOIN keywords
ON keywords.id = keywords_wine.keyword_id
--WHERE keywords.name IN ('coffee','toast','green apple', 'cream', 'citrus') AND (keywords_wine.count > 10)
--Adding a lot of ands, so wines have ALL the keywords
--WHERE (keywords.name = 'coffee') AND (keywords.name = 'toast') AND (keywords.name = 'green apple') AND (keywords.name = 'cream') AND (keywords.name = 'citrus')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


We would like to do a selection of wines that are easy to find all over the world. Find the top 3 most common grape all over the world and for each grape, give us the the 5 best rated wines.

In [14]:
%%sql --COMPLETE
SELECT grapes.name AS grape_name, grapes.id AS grape_id, SUM(most_used_grapes_per_country.wines_count) AS total_world_wide
FROM most_used_grapes_per_country
JOIN grapes
ON grapes.id = most_used_grapes_per_country.grape_id
GROUP BY most_used_grapes_per_country.wines_count
ORDER BY most_used_grapes_per_country.wines_count DESC
LIMIT 3

 * sqlite:///vivino_v2.db
Done.


grape_name,grape_id,total_world_wide
Cabernet Sauvignon,2,9621012
Chardonnay,5,3625248
Pinot Noir,14,1717002


We would to give create a country leaderboard, give us a visual that shows the average wine rating for each country. Do the same for the vintages.

In [20]:
%%sql -- WITHOUT VINTAGES
SELECT DISTINCT countries.name as country, ROUND(AVG(wines.ratings_average),2) AS average_rating
from countries
JOIN regions
ON countries.code = regions.country_code
JOIN wines
ON wines.region_id = regions.id
GROUP BY countries.name
ORDER BY average_rating DESC


 * sqlite:///vivino_v2.db
Done.


country,average_rating
Israël,4.5
Allemagne,4.5
États-Unis,4.49
Moldavie,4.48
Hongrie,4.47
Australie,4.46
Afrique du Sud,4.46
France,4.45
Portugal,4.44
Espagne,4.44


In [16]:
%%sql -- INCLUDED VINTAGES
SELECT DISTINCT countries.name as country, ROUND(AVG(vintages.ratings_average),2) AS average_ratings_vintage
from countries
JOIN regions
ON countries.code = regions.country_code
JOIN wines
ON wines.region_id = regions.id
-- adding vintages
JOIN vintages
ON vintages.wine_id = wines.id
GROUP BY countries.name
ORDER BY average_ratings_vintage DESC


 * sqlite:///vivino_v2.db
Done.


country,average_ratings_vintage
Roumanie,4.5
Croatie,4.4
Argentine,4.32
Chili,4.12
Portugal,3.93
Moldavie,3.87
États-Unis,3.7
Afrique du Sud,3.66
Italie,3.63
Espagne,3.38


Give us any other useful insights you found in our data. Be creative!

**Nice-to-have features**

One of our VIP client like Cabernet Sauvignon, he would like a top 5 recommandation, which wines would you recommend to him?

In [17]:
%%sql--WITH GRAPE
SELECT wines.id AS wine_id, wines.name
FROM wines
JOIN keywords_wine
ON keywords_wine.wine_id = wines.id
JOIN keywords
ON keywords_wine.keyword_id = keywords.id
WHERE keywords_wine.keyword_id = 2
LIMIT 5

UsageError: Cell magic `%%sql--WITH` not found.


## Plotting